In [59]:
from pyfuseki import FusekiQuery
import pysolr  

#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/acervo/', timeout=10)

In [1]:
from api.src.schemas.settings import Settings

settings = Settings()

In [3]:
collection = FusekiQuery(f'{settings.url}:3030', 'collection')


In [19]:
def GenerateId(base):

    query = FusekiQuery(f'{settings.url}:3030', base)

    sparql = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            SELECT ?g
            {graph ?g {?s bf:generationDate ?o}} 
            ORDER BY DESC(?o)
            LIMIT 1"""
    
    response = query.run_sparql(sparql)
    response = response.convert()
    bindings = response['results']['bindings']

    if len(bindings) == 0:
        return { 'id': 1 }
    else:
        value = bindings[0]['g']['value']
        bk = int(value.split("/")[1]) + 1
        return { 'id': "bkc-"+str(bk) } 
    
id = GenerateId("collection")
id


{'id': 1}

In [20]:
id = GenerateId("authority")
id

{'id': 1}

In [17]:
query = FusekiQuery(f'{settings.url}:3030', "authority")

sparql = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            SELECT ?g
            {graph ?g {?s bf:generationDate ?o}} 
            ORDER BY DESC(?o)
            LIMIT 1"""
    
response = query.run_sparql(sparql)
response = response.convert()
bindings = response['results']['bindings']
bindings


[{'g': {'type': 'uri',
   'value': 'https://bibliokeia.com/authorities/PersonalName/bkau-2'}}]

In [18]:
value = bindings[0]['g']['value']
value

'https://bibliokeia.com/authorities/PersonalName/bkau-2'

TEST JORNEY OF HUMANITY

In [12]:
import httpx

Assunto

In [24]:
# Import LOC
url = 'http://localhost:8000/import/loc/subjects?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fsubjects%2Fsh85040817'
response = httpx.get(url)
response

<Response [200 OK]>

In [30]:
uri = 'http://id.loc.gov/authorities/subjects/sh85040804'
url = f'http://localhost:8000/import/loc/subjects?uri={uri}'
response = httpx.get(url, timeout=120)
response

<Response [200 OK]>

In [31]:
obj = response.json()

In [32]:
# POST ASSUNTO
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=obj)
response.status_code

201

Agent

In [34]:
url = 'http://localhost:8000/import/loc/agents?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fnames%2Fn86870523'
response = httpx.get(url, timeout=120)
response


<Response [200 OK]>

WORK

In [48]:
url = 'http://localhost:8000/import/loc/works?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Fworks%2F22600263'
response = httpx.get(url, timeout=120)
response

<Response [200 OK]>

In [49]:
response.json()

{'adminMetadata': {'encodingLevel': 'menclvl:f',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-28',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-06-28T15:01:46.153359',
  'identifiedBy': [{'type': 'Local',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': '22600263'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'type': ['Work', 'Text', 'Monograph'],
 'content': [{'label': 'text',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt',
   'type': ['http://id.loc.gov/ontologies/bibframe/Content']}],
 'language': [{'label': 'English',
   'lang': 'en',
   'uri': 'http://id.loc.gov/vocabulary/languages/eng',
   'type': ['ht

In [50]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.contribution import Contribution
from api.src.schemas.metadata.bibframe.element import Element
from api.src.schemas.metadata.bibframe.title import Title

class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    content: list[Element]
    language: list[Element]
    title: Title
    classification: Optional[Classification]
    contribution: Optional[list[Contribution]]
    subject: Optional[list[Element]]   
    genreForm: Optional[list[Element]]
    note: Optional[str]
    summary: Optional[str]
    tableOfContents: Optional[str]
    supplementaryContent: Optional[list[Element]]
    illustrativeContent: Optional[list[Element]]
    intendedAudience: Optional[list[Element]]
    geographicCoverage: Optional[list[Element]]

In [51]:
request = Work(**response.json())
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 28), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 28, 15, 1, 46, 153359), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='22600263')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text', 'Monograph'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language

In [57]:
obj = {
  "adminMetadata": {
    "encodingLevel": "menclvl:f",
    "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
    "creationDate": "2023-06-28",
    "descriptionConventions": "http://id.loc.gov/vocabulary/descriptionConventions/isbd",
    "descriptionModifier": "http://id.loc.gov/vocabulary/organizations/brmninpa",
    "descriptionLanguage": "http://id.loc.gov/vocabulary/languages/por",
    "generationProcess": "BiblioKeia v.1",
    "generationDate": "2023-06-28T15:09:59.546860",
    "identifiedBy": [
      {
        "type": "Local",
        "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
        "value": "22600263"
      }
    ],
    "status": {
      "value": "mstatus:new",
      "label": "novo"
    }
  },
  "type": [
    "Work",
    "Text",
    "Monograph"
  ],
  "content": [
    {
      "label": "text",
      "uri": "http://id.loc.gov/vocabulary/contentTypes/txt",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Content"
      ]
    }
  ],
  "language": [
    {
      "label": "English",
      "lang": "en",
      "uri": "http://id.loc.gov/vocabulary/languages/eng",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Language",
        "http://www.loc.gov/mads/rdf/v1#Language"
      ]
    }
  ],
  "title": {
    "type": "bf:Title",
    "mainTitle": "The journey of humanity"
  },
  "classification": {
    "type": "ClassificationDdc",
    "classificationPortion": "338.9001",
    "edition": "full"
  },
  "contribution": [
    {
      "type": [
        "http://id.loc.gov/ontologies/bflc/PrimaryContribution"
      ],
      "agent": "https://bibliokeia.com/authorities/PersonalName/bka-3",
      "label": "Galor, Oded, 1953-",
      "role": "http://id.loc.gov/vocabulary/relators/aut"
    }
  ],
  "subject": [
    {
      "label": "Desenvolvimento econômico",
      "lang": "pt",
      "uri": "https://bibliokeia.com/authorities/Topic/bka-2",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Topic"
      ]
    },
    {
      "label": "História econômica",
      "lang": "pt",
      "uri": "https://bibliokeia.com/authorities/Topic/bka-1",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Topic"
      ]
    },
    {
      "label": "Equality",
      "lang": "en",
      "uri": "http://id.loc.gov/authorities/subjects/sh85044503",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Topic"
      ]
    }
  ],
  "genreForm": [
    {
      "label": "Informational works",
      "lang": "en",
      "uri": "http://id.loc.gov/authorities/genreForms/gf2014026113",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/GenreForm"
      ]
    }
  ],
  "summary": "\"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor's gripping narrative explains how technology, population size, and adaptation led to a stunning \"phase change\" in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also at an explanation of inequality's ultimate causes: those ancestral populations that enjoyed fruitful geographical characteristics and rich diversity were set on the path to prosperity, while those that lacked it were disadvantaged in ways still echoed today. As we face ecological crisis across the globe, The Journey of Humanity is a book of urgent truths and enduring relevance, with lessons that are both hopeful and profound: gender equality, investment in education, and balancing diversity with social cohesion are the keys not only to our species' thriving, but to its survival.\" -- publisher's website.",
  "tableOfContents": "Mysteries of the human journey -- [Part] I: The human odyssey. First steps. Genesis ; Exodus from the cradle of humankind ; Early settlement ; The dawn of civilisation -- Lost in stagnation. The Malthusian thesis ; The inevitable onset of agriculture ; Population swings ; The economic ice age -- The storm beneath the surface. Unified growth theory ; Wheels of change -- Full steam. Acceleration of technological development ; Education in the pre-industrial era ; Industrialisation and human capital ; The advent of universal public education ; Child labour no more -- Metamorphosis. Triggers of the demographic transition ; Family tales ; Phase transition -- The promised land. The twilight of industry ; The age of growth ; Growth and environmental degradation -- Coda: resolving the mystery of growth -- [Part] II: The origins of wealth and inequality. Splendour and misery. Disparate factors ; Rusty tools ; Trade, colonialism and uneven development ; Deep-rooted factors -- The fingerprints of institutions. Institutional origins of the British ascent ; Institutions and long-run development ; The legacy of colonialism ; Origins of institutions -- The cultural factor. The power of culture ; A culture of growth ; Cultural inertia ; Culture and prosperity -- The shadow of geography. Landscape fragmentation and the rise of Europe ; Origins of extractive institutions ; Geographical roots of cultural traits ; Roots of comparative development -- The legacy of the agricultural revolution. Roots and impacts of the Neolithic revolution ; The grains of civilisation ; Ceding the head start ; Geography's decree -- Out of Africa. Origins of human diversity ; Measuring diversity ; Diversity and prosperity ; The grip of the past -- Coda: unravelling the mystery of inequality -- Afterword.",
  "supplementaryContent": [
    {
      "label": "bibliography",
      "uri": "http://id.loc.gov/vocabulary/msupplcont/bibliography",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/SupplementaryContent"
      ]
    },
    {
      "label": "index",
      "uri": "http://id.loc.gov/vocabulary/msupplcont/index",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/SupplementaryContent"
      ]
    }
  ],
  "illustrativeContent": [
    {
      "label": "Illustrations",
      "uri": "http://id.loc.gov/vocabulary/millus/ill",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Illustration"
      ]
    },
    {
      "label": "Maps",
      "uri": "http://id.loc.gov/vocabulary/millus/map",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Illustration"
      ]
    },
    {
      "label": "Charts",
      "uri": "http://id.loc.gov/vocabulary/millus/chr",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Illustration"
      ]
    }
  ],
  "intendedAudience": [
    {
      "label": "Adult",
      "uri": "http://id.loc.gov/vocabulary/maudience/adu",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/IntendedAudience"
      ]
    }
  ]
}

In [58]:
request = Work(**obj)
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 6, 28), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 6, 28, 15, 9, 59, 546860), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='22600263')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text', 'Monograph'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language

In [ ]:
def DocWork(request, id):

    doc = {
        "id": id,
        "type": request.type,
        "content": [i.label for i in request.content],
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.classificationPortion,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        }
    
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = { "id": f"{id}/contribution/{i.agent.split('/')[-1]}",
                "type": [i.split('/')[-1] for i in i.type],
                "agent": i.agent,
                "label": i.label,
                "role": i.role } 
            contributions.append(c)
        doc['contribution'] = contributions   

    if request.subject:
        pass
    if request.genreForm:
        pass

    responseSolr =  solr.add([doc], commit=True)

    return responseSolr 

responseSolr = DocWork(request, 'bkc-1')
responseSolr

In [80]:
if request.subject:
    subjects = list()
    for i in request.subject:
        s = { "id": f"{id}/subject/{i.uri.split('/')[-1]}",
                "type": [i.split('/')[-1] for i in i.type],
                "uri": i.uri,
                "label": i.label} 
        subjects.append(s)
    doc['subject'] = subjects


{'id': 'bkc-1/subject/bka-2', 'type': ['Topic'], 'uri': 'https://bibliokeia.com/authorities/Topic/bka-2', 'label': 'Desenvolvimento econômico'}
{'id': 'bkc-1/subject/bka-1', 'type': ['Topic'], 'uri': 'https://bibliokeia.com/authorities/Topic/bka-1', 'label': 'História econômica'}
{'id': 'bkc-1/subject/sh85044503', 'type': ['Topic'], 'uri': 'http://id.loc.gov/authorities/subjects/sh85044503', 'label': 'Equality'}


In [74]:
request
id = 'bkc-1'
doc = {
        "id": id,
        "type": request.type,
        "content": [i.label for i in request.content],
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.classificationPortion,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        }

if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = { "id": f"{id}/contribution/{i.agent.split('/')[-1]}",
                "type": [i.split('/')[-1] for i in i.type],
                "agent": i.agent,
                "label": i.label,
                "role": i.role } 
            contributions.append(c)
        doc['contribution'] = contributions  

In [75]:
doc

{'id': 'bkc-1',
 'type': ['Work', 'Text', 'Monograph'],
 'content': ['text'],
 'mainTitle': 'The journey of humanity',
 'language': ['English'],
 'subtitle': None,
 'cdd': '338.9001',
 'note': None,
 'summary': '"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor\'s gripping narrative explains how technology, population size, and adaptation led to a stunning "phase change" in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also at an explanation of inequality\'s ultimate causes: those ancestral populations that enjoyed frui

In [76]:
solr.add([doc], commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":153}}\n'

In [70]:
doc

{'id': 'bkc-1',
 'type': ['Work', 'Text', 'Monograph'],
 'content': ['text'],
 'mainTitle': 'The journey of humanity',
 'language': ['English'],
 'subtitle': None,
 'cdd': Classification(type='ClassificationDdc', classificationPortion='338.9001', itemPortion=None, edition='full'),
 'note': None,
 'summary': '"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor\'s gripping narrative explains how technology, population size, and adaptation led to a stunning "phase change" in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also

In [71]:
import json
j = json.load(doc)

AttributeError: 'dict' object has no attribute 'read'